In [ ]:
from flask import Flask, jsonify, request, render_template_string
import time

# Initialize the Flask app
app = Flask(__name__)

# Define a hardcoded password (in a real app, store securely or hash it)
PASSWORD = "mysecurepassword"

# Simulated tank level data
TANK_DATA = {
    "tank_level": 75,
    "status": "OK",
    "last_updated": "2024-12-28T12:00:00Z"
}

HISTORY_DATA = [
    {"timestamp": "2024-12-25T12:00:00Z", "tank_level": 70, "status": "OK"},
    {"timestamp": "2024-12-24T12:00:00Z", "tank_level": 80, "status": "OK"}
]

# HTML template for the main page with improved design
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tank Monitoring System</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #e0f7fa;
            margin: 0;
            padding: 0;
        }

        header {
            background-color: #0288d1;
            color: white;
            text-align: center;
            padding: 1rem;
        }

        .container {
            width: 80%;
            margin: 0 auto;
            padding: 20px;
        }

        h1 {
            color: #333;
        }

        .status-section {
            margin-top: 20px;
            background-color: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }

        .status-section p {
            font-size: 18px;
            margin-bottom: 10px;
        }

        .status-section span {
            font-weight: bold;
            color: #333;
        }

        .form-container {
            background-color: #fff;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            margin-bottom: 20px;
        }

        input[type="password"] {
            padding: 8px;
            width: 60%;
            margin-right: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
        }

        button {
            padding: 10px 20px;
            background-color: #0288d1;
            color: white;
            border: none;
            border-radius: 4px;
            cursor: pointer;
        }

        button:hover {
            background-color: #e0f7fa;
        }

        footer {
            text-align: center;
            margin-top: 40px;
            padding: 10px;
            background-color: #0288d1;
            color: white;
        }

    </style>
</head>
<body>
    <header>
        <h1>Tank Monitoring System</h1>
    </header>
    
    <div class="container">
        <div class="form-container">
            <h2>Enter the password to view history:</h2>
            <form action="/history" method="POST">
                <input type="password" name="password" placeholder="Enter password" required>
                <button type="submit">Submit</button>
            </form>
        </div>
        
        <div class="status-section">
            <h2>Current Tank Status</h2>
            <p>Current Tank Level: <span id="tankLevel">Loading...</span></p>
            <p>Status: <span id="status">Loading...</span></p>
            <p>Last Updated: <span id="lastUpdated">Loading...</span></p>
        </div>
    </div>

    <footer>
        <p>&copy; 2024 Tank Monitoring System. All Rights Reserved.</p>
    </footer>

    <script>
        function fetchTankData() {
            fetch('/tank-level')
                .then(response => response.json())
                .then(data => {
                    document.getElementById('tankLevel').innerText = data.tank_level + '%';
                    document.getElementById('status').innerText = data.status;
                    document.getElementById('lastUpdated').innerText = data.last_updated;
                })
                .catch(error => console.error('Error fetching tank data:', error));
        }

        setInterval(fetchTankData, 5600); // Poll every 5 seconds
    </script>
</body>
</html>
"""

# HTML template for the history page with improved design
HISTORY_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tank History</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f9;
            margin: 0;
            padding: 0;
        }

        header {
            background-color: #0288d1;
            color: white;
            text-align: center;
            padding: 1rem;
        }

        .container {
            width: 80%;
            margin: 0 auto;
            padding: 20px;
        }

        h1 {
            color: #333;
        }

        ul {
            list-style-type: none;
            padding: 0;
        }

        li {
            background-color: #fff;
            padding: 15px;
            margin: 10px 0;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }

        li b {
            color: #0288d1;
        }

        a {
            color: #0288d1;
            text-decoration: none;
            font-size: 16px;
        }

        a:hover {
            text-decoration: underline;
        }

        footer {
            text-align: center;
            margin-top: 40px;
            padding: 10px;
            background-color: #0288d1;
            color: white;
        }
    </style>
</head>
<body>
    <header>
        <h1>Tank History</h1>
    </header>

    <div class="container">
        <ul>
            {% for record in history %}
            <li>
                <b>{{ record.timestamp }}</b>: Level {{ record.tank_level }}% (Status: {{ record.status }})
            </li>
            {% endfor %}
        </ul>
        <a href="/">Go Back</a>
    </div>

    <footer>
        <p>&copy; 2024 Tank Monitoring System. All Rights Reserved.</p>
    </footer>
</body>
</html>
"""

@app.route("/", methods=["GET"])
def home():
    """Render the main page with a password box."""
    return render_template_string(HTML_TEMPLATE)

@app.route("/history", methods=["POST"])
def history():
    """Validate the password and display the history."""
    password = request.form.get("password")
    if password == PASSWORD:
        return render_template_string(HISTORY_TEMPLATE, history=HISTORY_DATA)
    else:
        return """
        <h1>Unauthorized</h1>
        <p>Incorrect password. <a href="/">Try again</a>.</p>
        """, 401

@app.route("/update-tank", methods=["POST"])
def update_tank():
    """Endpoint to accept POST data."""
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No JSON payload received"}), 400
        
        # Log received data (for debugging)
        print(f"Received data: {data}")
        
        # Update TANK_DATA
        TANK_DATA.update(data)
        TANK_DATA["last_updated"] = time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())
        
        # Append to HISTORY_DATA
        HISTORY_DATA.append({
            "timestamp": TANK_DATA["last_updated"],
            "tank_level": data.get("tank_level", TANK_DATA["tank_level"]),
            "status": data.get("status", TANK_DATA["status"]),
        })
        
        # Log updated data
        print(f"Updated tank data: {TANK_DATA}")
        
        return jsonify({"message": "Tank data updated successfully", "current_data": TANK_DATA}), 200
    except Exception as e:
        print(f"Error processing data: {e}")
        return jsonify({"error": "Failed to process data"}), 500

@app.route("/tank-level", methods=["GET"])
def tank_level():
    """API endpoint to get current tank level data."""
    return jsonify(TANK_DATA)

# Run the app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)
